<p><font size="6"><b> Basis Raster operaties en raster-vector tools</b></font></p>

> *GCCA+ phase 2 - Geopyhton training*  
> *June, 2023*
>
> *© 2023, Jasper Feyen  (<mailto:jasperfeyen@hotmail.com>)*

---
---

In het vorige notebook focusten we enkel op vectoren of raster. Uiteraard kunnen we veel meer indien we vectoren en rasters kunnen combinerenç

In [ ]:
import pandas as pd
import numpy as np
import geopandas
import rasterio

import xarray as xr

import matplotlib.pyplot as plt

# `rioxarray`: de kracht van xarray en rasterio samen

In de vorige notebooks hebben we al gebruik gemaakt van `rasterio` (https://rasterio.readthedocs.io/en/latest/) om rasterbestanden zoals GeoTIFFs te lezen (via de `xarray.open_dataarray(...,engine="rasterio")` functie). Rasterio biedt ondersteuning voor het lezen en schrijven van georuimtelijke rastergegevens als numpy N-D arrays, voornamelijk via bindings met de GDAL-library.

Daarnaast biedt rasterio een Python API om enkele GIS-rasterbewerkingen uit te voeren (clipping, masking, warping, samenvoegen, transformatie, enz.) en kan het worden gebruikt om slechts een subset van een groot dataset in het geheugen te laden. De belangrijkste complexiteit bij het gebruik van `rasterio` is echter dat de ruimtelijke informatie losgekoppeld is van de gegevens zelf (d.w.z. de numpy-array). Dit betekent dat je de omvang en metadata gedurende de bewerkingen moet bijhouden en organiseren (bijv. de "transform") en dat je moet bijhouden wat elke dimensie vertegenwoordigt (y-eerst, aangezien arrays eerst georganiseerd zijn langs rijen). 

Maak kennis met `rioxarray` (https://corteva.github.io/rioxarray/stable/index.html), dat xarray uitbreidt met georuimtelijke functionaliteiten aangedreven door rasterio.


In [ ]:
import rioxarray

In [ ]:
data_file = "data/Sentinel_2022_example.tif"

In [ ]:
data = rioxarray.open_rasterio(data_file)
data

De `rioxarray.open_rasterio` functie is vergelijkbaar met `xarray.open_dataarray`.

Zodra `rioxarray` is geïmporteerd, biedt het een `.rio` accessor op het `xarray.DataArray` object, waarmee toegang wordt verkregen tot enkele eigenschappen van de rastergegevens:

In [ ]:
data.rio.crs

In [ ]:
data.rio.bounds()

In [ ]:
data.rio.resolution()

In [ ]:
data.rio.nodata

In [ ]:
data.rio.transform()

## Bandnamen hernoemen

Bij het bekijken van de raster dataset, hebben we de 10 verschillende Sentinel-2 banden, deze zijn genummerd van 1 tot 10. Dit komt niet overeen met de werkelijke bandnummer van Sentinel-2 zoals die moet gegeven zijn.

Zo is onze band 1 de blauwe band, wat volgens Sentinel-2 benaming eigenlijk "Band 2" moet zijn. Zo klopt voor elk van onze banden het gegeven bandnummer niet met de bandbenamingen. Kijken we onder de 'Attributes', zien we echter wel dat de 'long name' als metadata zit opgeslagen: de werkelijke bandnamen.

Het zou dus handig zijn om deze labels toe te kennen aan onze banden, om verwarring te voorkomen. Gelukkig kan dit!

In [ ]:
# Bandnamen zitten opgeslagen onder de attributen
data.attrs['long_name']

We kunnen de labels toekennen met assign_coordinates:

In [ ]:
data = data.assign_coords(band=("band", list(data.attrs['long_name'])))
data

## Reprojecting rasters

`rioxarray` geeft toegang tot een set rasterverwerkingsfuncties van rasterio/GDAL.

Een daarvan is het herschalen (transformeren en resampling) van rasters, bijvoorbeeld om naar een ander coördinatenreferentiesysteem te transformeren, naar een andere resolutie te verkleinen/vergroten, enzovoort. In al deze gevallen, bij de transformatie van een oorspronkelijk raster naar een doelraster, moeten pixelwaarden opnieuw worden berekend. Er zijn verschillende "resampling"methoden die hiervoor kunnen worden gebruikt: de Nearest Neighbor pixelwaarde, het gemiddelde berekenen, een (niet-)lineaire interpolatie, enzovoort..

De functionaliteit is beschikbaar via de `reproject()`-methode.:


In [ ]:
#We nemen enkel de rode, groene en blauwe band (= B4, B3, B2)
data.sel(band = ['B4','B3','B2']).rio.crs

In [ ]:
data.sel(band = ['B4','B3','B2']).rio.reproject("EPSG:31370").plot.imshow(figsize=(10,6),vmax=0.2)

De *default* resampling methode  is "nearest", wat vaak geen geschikte methode is (vooral voor continue data). We kunnen de methode wijzigen met behulp van de `rasterio.enums.Resampling`-enumeratie (zie [docs](https://rasterio.readthedocs.io/en/latest/api/rasterio.enums.html#rasterio.enums.Resampling) voor een overzicht van alle methoden):

In [ ]:
from rasterio.enums import Resampling

In [ ]:
data.rio.reproject("EPSG:31370", resampling=Resampling.bilinear).sel(band=['B4','B3','B2']).plot.imshow(figsize=(10,6),vmax = 0.2)

Je kunt ook meteen downsamplen

In [ ]:
data.rio.reproject(data.rio.crs, resolution=120, resampling=Resampling.cubic).sel(band=['B4','B3','B2']).plot.imshow(figsize=(10,6),vmax = 0.2)

# Clip raster by mask layer

Binnen veel toepassingen is het studiegebied (veel) kleiner dan de gegeven rasters.

In [ ]:
districts = geopandas.read_file("./data/Suriname_districts.geojson")
paramaribo_district = districts[districts['DISTR_NAAM'] == 'Paramaribo']
paramaribo_district

In [ ]:
paramaribo_district.plot()

In [ ]:
paramaribo_district.crs

We moeten zorgen dat beide datasets in hetzelfde CRS staan!

In [ ]:
paramaribo_district = paramaribo_district.to_crs(epsg=32621)

In [ ]:
clipped = data.rio.clip(paramaribo_district.geometry)

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(12,4))
data.sel(band=['B4','B3','B2']).plot.imshow(ax=ax0, vmax = 0.2)
paramaribo_district.plot(ax=ax0, facecolor="none", edgecolor="red")

clipped.sel(band=['B4','B3','B2']).plot.imshow(ax=ax1, vmax = 0.2)
paramaribo_district.plot(ax=ax1, facecolor="none", edgecolor="red")

fig.tight_layout()

De bovenstaande code maakt gebruik van het `rasterio`-pakket (met de functionaliteit `mask` en `geometry_mask` / `rasterize`) onder de motorkap. Dit vereenvoudigt de bewerking in vergelijking met het direct gebruik van `rasterio`.

```python
   # cfr. The Rasterio workflow

   from rasterio.mask import mask

   # 1 - Open een dataset met behulp van de contextbeheerder
   with rasterio.open(data_file) as src:

    # 2 - Lees de dataset en transformeer deze door uit te snijden
    out_image, out_transform = mask(src, paramaribo_vect.geometry, crop=True)

    # 3 - Werk de ruimtelijke metadata/profiel van de dataset bij
    paramaribo_profile = src.profile
    paramaribo_profile.update({"height": out_image.shape[1],
                              "width": out_image.shape[2],
                              "transform": out_transform})
    # 4 - Sla de nieuwe dataset op met de bijgewerkte metadata/profiel
    with rasterio.open("./paramaribo_masked.tif", "w", **paramaribo_profile) as dest:
        dest.write(out_image)



# Raster bewerkingen

## Reducties & element-gewijze berekeningen

In [ ]:
paramaribo = xr.open_dataarray( "data/Sentinel_2022_example.tif", engine="rasterio")
paramaribo = paramaribo.assign_coords(band=("band", list(paramaribo.attrs['long_name'])))
paramaribo_red = paramaribo.sel(band="B4")

### Reducties

De __reductie__ (aggregaties) worden aangeboden als methoden en kunnen worden toegepast langs één of meerdere dimensies van de gegevens.

Standaard wordt een array gereduceerd over *alle* dimensies, waarbij een enkele waarde als ouput wordt gegeven als een DataArray:

In [ ]:
paramaribo_red.mean()

In NumPy, deze dimenties worden __axis__ genoemd:

In [ ]:
paramaribo_red.mean(axis=1)

Maar we hebben dimensies met labels (zoals 'X', en 'Y'), dus in plaats van reducties uit te voeren op assen (zoals in NumPy), kunnen we ze uitvoeren op de dimensies (x of y as). Dit is handiger:

In [ ]:
paramaribo_red.mean(dim="x")

Berekenen van gemiddelde per as

In [ ]:
paramaribo.mean(dim=["x", "y"])  #  'neem het gemiddelde over de x- en y- as samen)

### Elementsgewijze berekeningen

In Xarray kun je gemakkelijk elementswijze berekingen uitvoeren. In onderstaand voorbeeld vermenigvuldig je dus **elke** pixel in de rode band met 10

In [ ]:
paramaribo_red * 10.

Indien we meerdere banden met elkaar combineren, gebeurt ook dit elementgewijs; 

In [ ]:
paramaribo.sel(band="B4") - paramaribo.sel(band="B3")

### Oefenen dan maar!

<div class="alert alert-success">

**OEFENING**:

Op basis van onze verschillende banden in het Sentinel-beeld, kunnen we de NDVI berekenen. Deze wordt berekend op basis van band 8 en band 4:
    
$$\frac{band_8 - band_4}{band_8 + band_4} $$
    
Maak gebruik van de Paramaribo-set:
    
- Zorg er eerst voor dat de verschillende banden hun correcte naam meekrijgen. 
- Bereken de NDVI, sla deze op als een nieuwe variabele: paramaribo_ndvi
- Maak een visualitie. Gebruik "Greens" als colormap. 


           
</div>

In [ ]:
paramaribo

In [ ]:
ndvi = (paramaribo.sel(band = 'B8') - paramaribo.sel(band = 'B4'))/(paramaribo.sel(band = 'B8') + paramaribo.sel(band='B4'))

In [ ]:
ndvi.plot.imshow(figsize=(10,6),cmap = 'Greens',vmin=0.5,vmax=1)

### Convert vector to raster

To create a raster with the vector "burned in", we can use the `rasterio.features.rasterize` function. This expects a list of (shape, value) tuples, and an output image shape and transform. Here, we will create a new raster image with the same shape and extent as the DEM above. And we first take a buffer of the river lines:

In [ ]:
import rasterio.features

In [ ]:
segments_buffered = segments.geometry.buffer(100)
img = rasterio.features.rasterize(
    segments_buffered, 
    out_shape=clipped.shape, 
    transform=clipped.rio.transform())

In [ ]:
img

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(img)
ax1.imshow(clipped.values - img*20, vmin=0, cmap="terrain") # just as an example
fig.tight_layout()

# Raster statistieken

The **rasterstats** package provides methods to calculate summary statistics of geospatial raster datasets based on vector geometries (https://github.com/perrygeo/python-rasterstats)

Under c

To illustrate this, we are reading a raster file with elevation data of the full world (the file contains a single band for the elevation, save the file in the `data` subdirectory; [download link](https://www.eea.europa.eu/data-and-maps/data/world-digital-elevation-model-etopo5/zipped-dem-geotiff-raster-geographic-tag-image-file-format-raster-data/zipped-dem-geotiff-raster-geographic-tag-image-file-format-raster-data/at_download/file)):

In [ ]:
countries = geopandas.read_file("./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("./data/ne_110m_populated_places.zip")

In [ ]:
dem_geotiff = "data/DEM_geotiff/alwdgg.tif"

In [ ]:
img = xr.open_dataarray(dem_geotiff).sel(band=1).plot.imshow(cmap="terrain", figsize=(10, 4), )
img.axes.set_aspect("equal")

Given this raster of the elevation, we might want to know the elevation at a certain location or for each country.
For the countries example, we want to extract the pixel values that fall within a country polygon, and calculate a statistic for it, such as the mean or the maximum.

Such functionality to extract information from a raster for given vector data is provided by the rasterstats package.

In [ ]:
import rasterstats

For extracting the pixel values for polygons, we use the `zonal_stats` function, passing it the GeoSeries, the path to the raster file, and the method to compute the statistics.

In [ ]:
result = rasterstats.zonal_stats(countries.geometry, dem_geotiff,
                                 stats=['min', 'mean', 'max'], 
                                 nodata=-999)

The results can be assigned to new columns:

In [ ]:
countries[['min', 'max', 'mean']] = pd.DataFrame(result)

In [ ]:
countries.head()

And then we can sort by the average elevation of the country:

In [ ]:
countries.sort_values('mean', ascending=False).head()

For points, a similar function called `point_query` can be used (specifying the interpolation method):

In [ ]:
cities["elevation"] = rasterstats.point_query(cities.geometry, 
                                              dem_geotiff, interpolate='bilinear',
                                              nodata=-999)

In [ ]:
cities.sort_values(by="elevation", ascending=False).head()

Thanks to https://geohackweek.github.io/raster/04-workingwithrasters/ for the inspiration